## AutoGen Simple Tool

*[Coding along with the Udemy online course [Mastering AutoGen: Building Multi-Agent Systems](https://www.udemy.com/course/autogen-agent-systems/?couponCode=MTST7102224A2) by Paulo Dichone]*

AutoGen 2.0 for Python documentation @ https://microsoft.github.io/autogen/0.2/docs/Getting-Started

In [11]:
# https://microsoft.github.io/autogen/0.2/docs/reference/agentchat/conversable_agent/
from autogen import ConversableAgent, UserProxyAgent
import pandas as pd

In [12]:
from typing import Annotated

Importing `autogen` returned the following error:<br/>
`flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.`

The temporary solution to solve this was to create a constraints.txt file with the following content:

`flaml==2.2.0`

Then running: `pip install -c constraints.txt autogen`

Following the suggestion at https://github.com/microsoft/autogen/issues/3548.

*FLAML @ https://microsoft.github.io/FLAML/ is "A Fast Library for Automated Machine Learning & Tuning."*

In [13]:
api_key = pd.read_csv("~/tmp/chat_gpt/agentic-design-1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


In [14]:
llm_config = {
    "model": "gpt-4o-mini",
    "temperature": 0.0,
    "api_key": api_key
    }
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


In [15]:
# defining simple calculator functions
def add_numbers(
    a: Annotated[int, "First number"], b: Annotated[int, "Second number"]
) -> str:
    return f"The sum of {a} and {b} is {a + b}."

def multiply_numbers(
    a: Annotated[int, "First number"], b: Annotated[int, "Second number"]
) -> str:
    return f"The product of {a} and {b} is {a * b}."

In [16]:
assistant = ConversableAgent(
    name = "CalculatorAssistant",
    system_message = "You're a helpful AI calculator. Return 'TERMINATE' when the task is done.",
    llm_config = llm_config
)
assistant

In [17]:
user_proxy = ConversableAgent(
    name = "User",
    is_termination_msg = lambda msg: msg.get("content") is not None
    and "TERMINATE" in msg["content"],
    human_input_mode = "NEVER"
)
user_proxy

In [18]:
# register the tool signatures for the assistant agent
assistant.register_for_llm(name="add_numbers", description="Add two numbers.")(
    add_numbers
)

assistant.register_for_llm(name="multiply_numbers", description="Multiply two numbers.")(
    multiply_numbers
)

<function __main__.multiply_numbers(a: Annotated[int, 'First number'], b: Annotated[int, 'Second number']) -> str>

In [19]:
# registering the tool functions with the user proxy agent
user_proxy.register_for_execution(name="add_numbers")(
    add_numbers
)

user_proxy.register_for_execution(name="multiply_numbers")(
    multiply_numbers
)

<function __main__.multiply_numbers(a: Annotated[int, 'First number'], b: Annotated[int, 'Second number']) -> str>

In [20]:
user_proxy.initiate_chat(
    assistant,
    message = "What is the sum of 7 and 5?"
)

User (to CalculatorAssistant):

What is the sum of 7 and 5?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
CalculatorAssistant (to User):

***** Suggested tool call (call_e104c3s2aoE9rf1BwuEZGq1i): add_numbers *****
Arguments: 
{"a":7,"b":5}
****************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION add_numbers...
User (to CalculatorAssistant):

User (to CalculatorAssistant):

***** Response from calling tool (call_e104c3s2aoE9rf1BwuEZGq1i) *****
The sum of 7 and 5 is 12.
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
CalculatorAssistant (to User):

The sum of 7 and 5 is 12. 

TERMINATE

---------------------------------------------------------------------------

ChatResult(chat_id=None, chat_history=[{'content': 'What is the sum of 7 and 5?', 'role': 'assistant', 'name': 'User'}, {'tool_calls': [{'id': 'call_e104c3s2aoE9rf1BwuEZGq1i', 'function': {'arguments': '{"a":7,"b":5}', 'name': 'add_numbers'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': 'The sum of 7 and 5 is 12.', 'tool_responses': [{'tool_call_id': 'call_e104c3s2aoE9rf1BwuEZGq1i', 'role': 'tool', 'content': 'The sum of 7 and 5 is 12.'}], 'role': 'tool', 'name': 'User'}, {'content': 'The sum of 7 and 5 is 12. \n\nTERMINATE', 'role': 'user', 'name': 'CalculatorAssistant'}], summary='The sum of 7 and 5 is 12. \n\n', cost={'usage_including_cached_inference': {'total_cost': 6.03e-05, 'gpt-4o-mini-2024-07-18': {'cost': 6.03e-05, 'prompt_tokens': 262, 'completion_tokens': 35, 'total_tokens': 297}}, 'usage_excluding_cached_inference': {'total_cost': 6.03e-05, 'gpt-4o-mini-2024-07-18': {'cost': 6.03e-05, 'prompt_tokens': 262, 'completion_tokens': 35, 'total_tokens'